In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd

import nltk
from nltk.tokenize import TreebankWordTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 전처리 끝낸 데이터 불러오기
raw = pd.read_csv('/Users/mac/AIFFEL/hackathon/new_musinsa(upcycling).csv', engine='python')
print(raw.shape)
raw.head()

(715, 12)


,Unnamed: 0,user,color,contents,category,ratings,gender,price,style,up_check,brand,url
0,0,멤버_464f9cc84f7d,기타,면,상의,5,남자,39000,0,0,119레오,https://www.musinsa.com/app/goods/1876547/0
1,1,멤버_464f9cc84f7d,카키,면,하의,5,남자,42000,0,0,119레오,https://www.musinsa.com/app/goods/2396824/0
2,2,멤버_464f9cc84f7d,기타,면,모자,5,남자,59000,0,0,119레오,https://www.musinsa.com/app/goods/2428013/0
3,3,멤버_464f9cc84f7d,블랙,면,상의,5,남자,35000,0,0,119레오,https://www.musinsa.com/app/goods/470357/0
4,4,멤버_464f9cc84f7d,아이보리,면 아라미드,가방,5,남자,48000,1,1,119레오,https://www.musinsa.com/app/goods/1766332/0


In [3]:
# 불필요한 컬럼 제거
raw.drop(['Unnamed: 0'], axis=1, inplace=True)
raw.head()

,user,color,contents,category,ratings,gender,price,style,up_check,brand,url
0,멤버_464f9cc84f7d,기타,면,상의,5,남자,39000,0,0,119레오,https://www.musinsa.com/app/goods/1876547/0
1,멤버_464f9cc84f7d,카키,면,하의,5,남자,42000,0,0,119레오,https://www.musinsa.com/app/goods/2396824/0
2,멤버_464f9cc84f7d,기타,면,모자,5,남자,59000,0,0,119레오,https://www.musinsa.com/app/goods/2428013/0
3,멤버_464f9cc84f7d,블랙,면,상의,5,남자,35000,0,0,119레오,https://www.musinsa.com/app/goods/470357/0
4,멤버_464f9cc84f7d,아이보리,면 아라미드,가방,5,남자,48000,1,1,119레오,https://www.musinsa.com/app/goods/1766332/0


# 컬럼 합치기

In [4]:
orginal_data_size = len(raw)

# rating 2점 이상만 사용
raw = raw[raw['ratings']>=2]
filtered_data_size = len(raw)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 715, filtered_data_size: 714
Ratio of Remaining Data is 99.86%


In [5]:
cols = ['color', 'contents', 'category', 'ratings', 'gender', 'price', 'style']
raw['features'] = raw[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
raw.drop(cols, axis=1, inplace=True)
raw

,user,up_check,brand,url,features
0,멤버_464f9cc84f7d,0,119레오,https://www.musinsa.com/app/goods/1876547/0,기타 면 상의 5 남자 39000 0
1,멤버_464f9cc84f7d,0,119레오,https://www.musinsa.com/app/goods/2396824/0,카키 면 하의 5 남자 42000 0
2,멤버_464f9cc84f7d,0,119레오,https://www.musinsa.com/app/goods/2428013/0,기타 면 모자 5 남자 59000 0
3,멤버_464f9cc84f7d,0,119레오,https://www.musinsa.com/app/goods/470357/0,블랙 면 상의 5 남자 35000 0
4,멤버_464f9cc84f7d,1,119레오,https://www.musinsa.com/app/goods/1766332/0,아이보리 면 아라미드 가방 5 남자 48000 1
...,...,...,...,...,...
710,허싱싱,1,플리츠마마,https://www.musinsa.com/app/goods/2041227/0,기타 폴리에스터 폴리우레탄 가방 5 여자 59000 0
711,허싱싱,0,플리츠마마,https://www.musinsa.com/app/goods/2196725/0,아이보리 울 아크릴 모자 5 여자 35000 0
712,석재은,0,플리츠마마,https://www.musinsa.com/app/goods/2122170/0,베이지 아크릴 모자 5 여자 29000 0
713,석재은,0,플리츠마마,https://www.musinsa.com/app/goods/2064480/0,블랙 면 폴리에스터 상의 5 여자 138000 0


In [6]:
# 컬럼 순서 정리
raw = raw[['user', 'features', 'brand', 'up_check', 'url']]
raw.head()

,user,features,brand,up_check,url
0,멤버_464f9cc84f7d,기타 면 상의 5 남자 39000 0,119레오,0,https://www.musinsa.com/app/goods/1876547/0
1,멤버_464f9cc84f7d,카키 면 하의 5 남자 42000 0,119레오,0,https://www.musinsa.com/app/goods/2396824/0
2,멤버_464f9cc84f7d,기타 면 모자 5 남자 59000 0,119레오,0,https://www.musinsa.com/app/goods/2428013/0
3,멤버_464f9cc84f7d,블랙 면 상의 5 남자 35000 0,119레오,0,https://www.musinsa.com/app/goods/470357/0
4,멤버_464f9cc84f7d,아이보리 면 아라미드 가방 5 남자 48000 1,119레오,1,https://www.musinsa.com/app/goods/1766332/0


In [7]:
# 데이터 프레임 재정의
new_df = raw[['user', 'features', 'brand']]
new_df.head()

,user,features,brand
0,멤버_464f9cc84f7d,기타 면 상의 5 남자 39000 0,119레오
1,멤버_464f9cc84f7d,카키 면 하의 5 남자 42000 0,119레오
2,멤버_464f9cc84f7d,기타 면 모자 5 남자 59000 0,119레오
3,멤버_464f9cc84f7d,블랙 면 상의 5 남자 35000 0,119레오
4,멤버_464f9cc84f7d,아이보리 면 아라미드 가방 5 남자 48000 1,119레오


# 코사인유사도(비슷한 브랜드 찾기)

[참고사이트](https://github.com/yoonsungsong/TIL/blob/main/RecommenderSytem/TF-IDF_info_and_project.ipynb)

In [8]:
# TreebankWord 토크나이저 사용
tb_tokenizer = TreebankWordTokenizer()

# TF-IDF 벡터화
tfidf = TfidfVectorizer(tokenizer=tb_tokenizer.tokenize)

tfidf_matrix = tfidf.fit_transform(new_df['features'])

print(tfidf_matrix.shape)

(714, 244)


In [9]:
# 코사인 유사도 매트릭스
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(cosine_matrix.shape)

(714, 714)


In [10]:
np.round(cosine_matrix, 4)

array([[1.    , 0.2028, 0.4636, ..., 0.0499, 0.1976, 0.4046],
       [0.2028, 1.    , 0.1742, ..., 0.0412, 0.1035, 0.0543],
       [0.4636, 0.1742, 1.    , ..., 0.2094, 0.1076, 0.0564],
       ...,
       [0.0499, 0.0412, 0.2094, ..., 1.    , 0.1002, 0.1065],
       [0.1976, 0.1035, 0.1076, ..., 0.1002, 1.    , 0.3141],
       [0.4046, 0.0543, 0.0564, ..., 0.1065, 0.3141, 1.    ]])

In [11]:
# brand 와 id 를 매핑할 dictionary 를 생성
brand2id = {}
for i, c in enumerate(new_df['brand']): brand2id[i] = c

# id 와 brand 를 매핑할 dictionary 를 생성
id2brand = {}
for i, c in brand2id.items(): id2brand[c] = i

In [12]:
idx = id2brand['119레오'] # 119레오 의 id 추출
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유사도 및 인덱스를 추출
sim_scores

[(0, 0.22618854926142679),
 (1, 0.11847107726156553),
 (2, 0.12318260387124363),
 (3, 0.19763430295604373),
 (4, 0.06533139967833543),
 (5, 0.21573973748896424),
 (6, 0.12277659543125827),
 (7, 0.13514912708631527),
 (8, 0.12850244095609595),
 (9, 0.16807992009464295),
 (10, 0.20695143198974608),
 (11, 0.14765702315854842),
 (12, 0.09753251377344105),
 (13, 0.11863666354047546),
 (14, 0.06533139967833543),
 (15, 0.07732914480105259),
 (16, 0.31241325682009946),
 (17, 0.38112697046736815),
 (18, 0.12221669794306554),
 (19, 0.22098641562105176),
 (20, 0.17504308811714855),
 (21, 0.803493267014111),
 (22, 0.9118393136364745),
 (23, 0.2033769868040666),
 (24, 0.10001530665674951),
 (25, 0.14154336453647756),
 (26, 0.25372267685284694),
 (27, 0.7367897457906442),
 (28, 0.2507465594135083),
 (29, 0.21785881222480527),
 (30, 0.15330350198600437),
 (31, 0.18746838773506075),
 (32, 0.4435367834859046),
 (33, 0.17229346642175974),
 (34, 0.1253809725634783),
 (35, 0.3905663671654589),
 (36, 0.236

In [13]:
# 유사도가 높은 순수대로 정렬
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
sim_scores[0:10] # 상위 10개의 인덱스 유사도 추출

[(498, 1.0000000000000002),
 (638, 0.9494197904496875),
 (22, 0.9118393136364745),
 (394, 0.9067803530307393),
 (21, 0.803493267014111),
 (247, 0.803493267014111),
 (572, 0.803493267014111),
 (27, 0.7367897457906442),
 (691, 0.7079851730717274),
 (395, 0.7029252615711632)]

In [14]:
# 인덱스에 해당되는 brand가 무엇인지 모르기 때문에 아까 만들어둔 brand2id 딕셔너리에 적용해준다
sim_scores = [(brand2id[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('큐클리프', 1.0000000000000002),
 ('플리츠마마', 0.9494197904496875),
 ('119레오', 0.9118393136364745),
 ('오버랩업사이클', 0.9067803530307393),
 ('119레오', 0.803493267014111),
 ('누깍', 0.803493267014111),
 ('플리츠마마', 0.803493267014111),
 ('119레오', 0.7367897457906442),
 ('플리츠마마', 0.7079851730717274),
 ('오버랩업사이클', 0.7029252615711632)]

# 코사인 유사도(비슷한 유저 찾기)

In [15]:
# user 와 id 를 매핑할 dictionary 를 생성
user2id = {}
for i, c in enumerate(new_df['user']): user2id[i] = c

# id 와 user 를 매핑할 dictionary 를 생성
id2user = {}
for i, c in user2id.items(): id2user[c] = i

In [16]:
idx = id2user['박소금소금소금'] # 박소금소금소금 의 id 추출
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유사도 및 인덱스를 추출
sim_scores

[(0, 0.19205709076308053),
 (1, 0.10587235033345828),
 (2, 0.1100828328187651),
 (3, 0.24773178884941316),
 (4, 0.061224900718873154),
 (5, 0.18318498650535886),
 (6, 0.06824813075373383),
 (7, 0.07821833380520717),
 (8, 0.01618274761408911),
 (9, 0.1996708968763226),
 (10, 0.2458483917172502),
 (11, 0.08710165891097353),
 (12, 0.08716048427859246),
 (13, 0.14844094806015684),
 (14, 0.061224900718873154),
 (15, 0.015213247024643958),
 (16, 0.11477061914549214),
 (17, 0.15293711128678678),
 (18, 0.17038810069854143),
 (19, 0.05030654548625238),
 (20, 0.03984775738215354),
 (21, 0.22005846649930222),
 (22, 0.19589519662830684),
 (23, 0.24160212199235973),
 (24, 0.032675915891023616),
 (25, 0.19733232410751847),
 (26, 0.08365820304797777),
 (27, 0.1138805470885172),
 (28, 0.08057236014136288),
 (29, 0.07000454450742472),
 (30, 0.2137276900248019),
 (31, 0.22270346805222457),
 (32, 0.08992937761110056),
 (33, 0.03922181859537191),
 (34, 0.17479956616712974),
 (35, 0.04249483746836898),
 (3

In [17]:
# 유사도가 높은 순수대로 정렬
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
sim_scores[0:10] # 상위 10개의 인덱스 유사도 추출

[(213, 0.3075697398494499),
 (104, 0.30689973404603255),
 (699, 0.30689973404603255),
 (191, 0.2994447398579822),
 (423, 0.2994447398579822),
 (437, 0.29931335294625194),
 (395, 0.2973415399110205),
 (306, 0.2917254911245464),
 (382, 0.29023448194555096),
 (193, 0.29004499475919066)]

In [18]:
# 만들어 둔 딕셔너리에 적용해 user id 와 brand 확인
sim_scores = [(user2id[i], brand2id[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('J flora', '누깍', 0.3075697398494499),
 ('병789기', '누깍', 0.30689973404603255),
 ('jjunnyy', '플리츠마마', 0.30689973404603255),
 ('칸쵸쵸칸', '누깍', 0.2994447398579822),
 ('dailylooked', '오버랩업사이클', 0.2994447398579822),
 ('뉴비_3dc706f3', '카네이테이', 0.29931335294625194),
 ('ZZOOst', '오버랩업사이클', 0.2973415399110205),
 ('불꽃남자$', '오버랩업사이클', 0.2917254911245464),
 ('bon05', '오버랩업사이클', 0.29023448194555096),
 ('칸쵸쵸칸', '누깍', 0.29004499475919066)]

# 코사인 유사도(함수로 만들기)

In [19]:
def simil_brand():
    
    new_user = input('새로운 user 이름을 입력하세요 : ')
    new_feature = input('새로운 아이템의 feature를 입력하세요 : ')
    print('\n')
    
    # 데이터 마지막행에 추가
    new_data = {'user' : new_user, 'features' : new_feature}
    new_raw = raw.append(new_data, ignore_index = True)
    
    # 토큰화 & 벡터화
    tb_tokenizer = TreebankWordTokenizer()

    tfidf = TfidfVectorizer(tokenizer=tb_tokenizer.tokenize)

    tfidf_matrix = tfidf.fit_transform(new_raw['features'])
    
    # 코사인 매트릭스
    cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    cosine_matrix.shape

    np.round(cosine_matrix, 4)

    # brand 와 id 를 매핑할 dictionary 를 생성
    brand2id = {}
    for i, c in enumerate(new_raw['brand']): brand2id[i] = c

    # id 와 brand 를 매핑할 dictionary 를 생성
    id2brand = {}
    for i, c in brand2id.items(): id2brand[c] = i
        
    # user 와 id 를 매핑할 dictionary 를 생성
    user2id = {}
    for i, c in enumerate(new_raw['user']): user2id[i] = c

    # id 와 user 를 매핑할 dictionary 를 생성
    id2user = {}
    for i, c in user2id.items(): id2user[c] = i

    idx = id2user[new_user]
    sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유사도 및 인덱스를 추출
    
    # 유사도가 높은 순수대로 정렬
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
    
    # 만들어 둔 딕셔너리에 적용해 user id 와 brand 확인
    sim_scores = [(user2id[i], brand2id[i], score) for i, score in sim_scores[0:10]]
    
    # 결과 문구 작성
    sim_user = print(f"당신과 비슷한 유저는 {sim_scores[0][0], sim_scores[1][0], sim_scores[2][0]} 입니다.", "\n")
    
    item1 = raw[(raw.user == sim_scores[0][0]) & (raw.up_check == 1)]['url']
    item2 = raw[(raw.user == sim_scores[1][0]) & (raw.up_check == 1)]['url']
    item3 = raw[(raw.user == sim_scores[2][0]) & (raw.up_check == 1)]['url']
    
    sim_item1 = print("당신과 비슷한 유저인", sim_scores[0][0],"이(가) 구매한 업사이클링 아이템은", sim_scores[0][1], item1, "입니다.", "\n")
    sim_item2 = print("당신과 비슷한 유저인", sim_scores[1][0],"이(가) 구매한 업사이클링 아이템은", sim_scores[2][1], item2, "입니다.", "\n")
    sim_item3 = print("당신과 비슷한 유저인", sim_scores[2][0],"이(가) 구매한 업사이클링 아이템은", sim_scores[3][1], item3, "입니다.")

    return sim_user, sim_item1, sim_item2, sim_item3

In [20]:
simil_brand()

새로운 user 이름을 입력하세요 : 가나다라
새로운 아이템의 feature를 입력하세요 : 블랙 폴리에스터 하의 5 남자 42000 0


당신과 비슷한 유저는 ('멤버_2d7aa55d30bb', '멤버_464f9cc84f7d', '콩슈크림빵') 입니다. 

당신과 비슷한 유저인 멤버_2d7aa55d30bb 이(가) 구매한 업사이클링 아이템은 누깍 68    https://www.musinsa.com/app/goods/2093274/0
Name: url, dtype: object 입니다. 

당신과 비슷한 유저인 멤버_464f9cc84f7d 이(가) 구매한 업사이클링 아이템은 누깍 4    https://www.musinsa.com/app/goods/1766332/0
Name: url, dtype: object 입니다. 

당신과 비슷한 유저인 콩슈크림빵 이(가) 구매한 업사이클링 아이템은 플리츠마마 165    https://www.musinsa.com/app/goods/2490789/0
Name: url, dtype: object 입니다.


(None, None, None, None)